# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table('DisasterMessages',engine)
X = df['message']
y = df.drop(['id','message','original','genre'],axis=1)

In [3]:
pd.set_option('display.max_colwidth', -1)
X.head(10)

0    Weather update - a cold front from Cuba that could pass over Haiti                                                    
1    Is the Hurricane over or is it not over                                                                               
2    Looking for someone but no name                                                                                       
3    UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.                  
4    says: west side of Haiti, rest of the country today and tonight                                                       
5    Information about the National Palace-                                                                                
6    Storm at sacred heart of jesus                                                                                        
7    Please, we need tents and water. We are in Silo, Thank you!                                                           
8    I w

In [4]:
pd.set_option('display.max_colwidth', -1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize
    text_normalized = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) 
    
    # Tokenize text
    text_tokenized = word_tokenize(text_normalized)
    
    # Remove stop words
    words_no_stopwords = [w for w in text_tokenized if w not in stopwords.words("english")]
    
    # Lemmatize
    words_lemmed = [WordNetLemmatizer().lemmatize(w) for w in words_no_stopwords]
    
    return words_lemmed    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
randomforest = RandomForestClassifier()

pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(randomforest))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [91]:
model_pred = pipeline.predict(X_test)
x = 0
while x<36:
    print(y.columns[x])
    print(classification_report(y_test[y.columns[x]],model_pred[:,x]))
    print('')
    x+=1

related
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        40
        1.0       0.83      0.98      0.90       207
        2.0       0.00      0.00      0.00         3

avg / total       0.68      0.81      0.74       250


request
             precision    recall  f1-score   support

        0.0       0.48      0.44      0.46       105
        1.0       0.62      0.66      0.64       145

avg / total       0.56      0.56      0.56       250


offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       249
        1.0       0.00      0.00      0.00         1

avg / total       0.99      1.00      0.99       250


aid_related
             precision    recall  f1-score   support

        0.0       0.35      0.32      0.34        91
        1.0       0.63      0.67      0.65       159

avg / total       0.53      0.54      0.53       250


medical_help
             precision    recall  f1

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fcd9d31e8c8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [95]:
parameters = {
    'clf__estimator__n_jobs': [1,2,3],
    'clf__estimator__min_samples_leaf': [1,2,3],
}

cv = GridSearchCV(pipeline,parameters)

cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [97]:
model_pred = cv.predict(X_test)
x = 0
while x<36:
    print(y.columns[x])
    print(classification_report(y_test[y.columns[x]],model_pred[:,x]))
    print('')
    x+=1

related
             precision    recall  f1-score   support

        0.0       0.14      0.03      0.04        40
        1.0       0.83      0.97      0.89       207
        2.0       0.00      0.00      0.00         3

avg / total       0.71      0.81      0.75       250


request
             precision    recall  f1-score   support

        0.0       0.50      0.53      0.51       105
        1.0       0.64      0.61      0.62       145

avg / total       0.58      0.58      0.58       250


offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       249
        1.0       0.00      0.00      0.00         1

avg / total       0.99      1.00      0.99       250


aid_related
             precision    recall  f1-score   support

        0.0       0.36      0.29      0.32        91
        1.0       0.63      0.70      0.67       159

avg / total       0.53      0.55      0.54       250


medical_help
             precision    recall  f1

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [98]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.